In [16]:
!pip install pyvirtualdisplay


In [17]:
!apt-get install -y xvfb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [18]:
from pyvirtualdisplay import Display
import cv2
import mediapipe as mp

# Initialize virtual display
display = Display(visible=0, size=(1024, 768))
display.start()


In [19]:
import cv2
import mediapipe as mp

# Initialize MediaPipe hands and drawing utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands()

# Function to calculate Euclidean distance
def calculate_distance(x1, y1, x2, y2):
    return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

# Video capture from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process frame for hand landmarks
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks on the frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract fingertip coordinates
            h, w, c = frame.shape
            index_finger_tip = hand_landmarks.landmark[8]  # Index finger tip landmark
            thumb_tip = hand_landmarks.landmark[4]  # Thumb tip landmark

            # Convert normalized landmarks to pixel coordinates
            index_x, index_y = int(index_finger_tip.x * w), int(index_finger_tip.y * h)
            thumb_x, thumb_y = int(thumb_tip.x * w), int(thumb_tip.y * h)

            # Calculate distance between thumb and index finger for click detection
            distance = calculate_distance(index_x, index_y, thumb_x, thumb_y)

            # Example action: print gesture if fingers are close
            if distance < 40:
                print("Click gesture detected")

    # Display the processed frame
    cv2.imshow("AI Gesture Control", frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()
